In [1]:
import configparser
from datetime import datetime
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.types import TimestampType

from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1609569452306_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
    # get filepath to song data file
song_data = "s3://ganpati-hanuman/song_data/*/*/*/*.json"

    # read song data file
df = spark.read.json(song_data)
    
    # extract columns to create songs table
songs_table = df.select('song_id', 'title', 'artist_id', 'year', 'duration').dropDuplicates(['song_id'])

    # write songs table to parquet files partitioned by year and artist
songs_table.write.mode('overwrite').partitionBy("year", "artist_id").parquet("s3://ganpati-hanuman/output/songs_table.parquet")
print('File songs_table.parquet created')

    # extract columns to create artists table
artists_table = df.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude').dropDuplicates(['artist_id'])

    # write artists table to parquet files
artists_table.write.mode('overwrite').parquet("s3://ganpati-hanuman/output/artists_table.parquet")
print('File artists_table.parquet created')




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

File songs_table.parquet created
File artists_table.parquet created

In [8]:
# get filepath to log data file
log_data = "s3://ganpati-hanuman/log_data/*.json"
  

    # read log data file
df = spark.read.json(log_data)

    # filter by actions for song plays
df = df.filter(df.page == 'NextSong')

    # extract columns for users table
users_table = df.select(col('userId').alias('user_id'), col('firstName').alias('first_name'),
                            col('lastName').alias('last_name'), 'gender', 'level')\
                            .dropDuplicates(['user_id', 'level'])

    # write users table to parquet files
users_table.write.mode('overwrite').parquet("s3://ganpati-hanuman/output/users_table.parquet")
print('File users_table.parquet created')

    # create timestamp column from original timestamp column
get_timestamp = udf(lambda x : datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S'))
df = df.withColumn('start_time', get_timestamp(df.ts)) 
    
    # extract columns to create time table
time_table = df.select(df.start_time.alias('start_time'), \
                hour(df.start_time).alias('hour'),\
                dayofmonth(df.start_time).alias('day'),\
                weekofyear(df.start_time).alias('week'),\
                month(df.start_time).alias('month'),\
                year(df.start_time).alias('year'),
                dayofweek(df.start_time).alias('weekday')).dropDuplicates()
    
    # write time table to parquet files partitioned by year and month
time_table.write.mode('overwrite').partitionBy(['year','month']).parquet("s3://ganpati-hanuman/output/time.parquet")    
print('File time.parquet created')

    # read in song data to use for songplays table
song_data = "s3://ganpati-hanuman/song_data/*/*/*/*.json"
songs_df = spark.read.json(song_data)
    # created song view to write SQL Queries
songs_df.createOrReplaceTempView("songs_table")

         
     # created log view to write SQL Queries
log_data = "s3://ganpati-hanuman/log_data/*.json"
log_df = spark.read.json(log_data)     
log_df.createOrReplaceTempView("logs_table")
    
    # extract columns from joined song and log datasets to create songplays table 
songplays_table = spark.sql(""" select monotonically_increasing_id() as songplay_id, 
    timestamp(l.ts/1000) as start_time, 
    month(to_timestamp(l.ts/1000)) as month, 
    year(to_timestamp(l.ts/1000)) as year, 
    l.userId as user_id, l.level as level, 
    s.song_id as song_id, s.artist_id as artist_id, 
    l.sessionId as session_id, l.location as location, 
    l.userAgent as user_agent FROM 
    logs_table l JOIN songs_table s on l.artist = s.artist_name and l.song = s.title """)
    

    # write songplays table to parquet files partitioned by year and month
songplays_table.write.mode('overwrite').partitionBy("year", "month").parquet("s3://ganpati-hanuman/output/songplays_table.parquet")
print('File songplays_table.parquet created')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

